# Capstone — Minimal Viable Submission

This single notebook implements the **minimal** end-to-end pipeline required for the capstone:

- Minimal cleaning
- Required EDA plots (5)
- Baseline forecasting (RandomForest) for total daily sales (next 30 days)
- Simple store segmentation (KMeans)
- Save outputs for submission

Follow cells in order and run everything. Paths and outputs are saved under `/mnt/data/`.

In [ ]:
# Imports and paths
import pandas as pd, numpy as np, matplotlib.pyplot as plt, os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.cluster import KMeans
import pickle

DATA_PATH = '/mnt/data/Retail_Sales_Data_Unlox_(1)[1].csv'
OUTPUT_DIR = '/mnt/data/capstone_outputs'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print('Output dir:', OUTPUT_DIR)

In [ ]:
# Load dataset
df = pd.read_csv(DATA_PATH)
print('Rows, cols:', df.shape)
df.head()

In [ ]:
# Minimal cleaning (only what's necessary)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
# Remove rows with invalid dates (if any)
df = df.dropna(subset=['Date']).copy()
# Ensure numeric columns
for c in ['Unit_Price','Units_Sold','Total_Sales','Discount_Percentage','Revenue','Stock_On_Hand','Store_Rating']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')
# Fill small missing numeric values with 0 for simplicity
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
df[num_cols] = df[num_cols].fillna(0)
# Basic derived columns
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['Is_Weekend'] = df['DayOfWeek'].isin([5,6]).astype(int)

# Save cleaned sample
cleaned_path = os.path.join(OUTPUT_DIR, 'processed_sales_minimal.csv')
df.to_csv(cleaned_path, index=False)
print('Cleaned saved to', cleaned_path)
df.head()

In [ ]:
# EDA: create aggregate daily sales (Total_Sales) and required plots
daily = df.groupby('Date', as_index=False)['Total_Sales'].sum().sort_values('Date')
daily = daily.rename(columns={'Total_Sales':'Daily_Sales'})

# 1. Monthly sales trend
monthly = daily.copy()
monthly['Month'] = monthly['Date'].dt.to_period('M')
monthly_agg = monthly.groupby('Month')['Daily_Sales'].sum().reset_index()
plt.figure(figsize=(10,4)); plt.plot(monthly_agg['Month'].astype(str), monthly_agg['Daily_Sales']); plt.xticks(rotation=45)
plt.title('Monthly Sales Trend'); plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR,'plot_monthly_sales.png')); plt.close()

# 2. Category-wise sales (top 10)
cat_sales = df.groupby('Product_Category', as_index=False)['Total_Sales'].sum().sort_values('Total_Sales', ascending=False).head(10)
plt.figure(figsize=(8,4)); plt.bar(cat_sales['Product_Category'], cat_sales['Total_Sales']); plt.xticks(rotation=45)
plt.title('Top 10 Product Categories by Sales'); plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR,'plot_category_sales.png')); plt.close()

# 3. Region-wise sales
region_sales = df.groupby('Region', as_index=False)['Total_Sales'].sum()
plt.figure(figsize=(6,4)); plt.bar(region_sales['Region'], region_sales['Total_Sales']); plt.title('Region-wise Sales'); plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR,'plot_region_sales.png')); plt.close()

# 4. Promotion vs Non-promotion sales
promo = df.copy()
promo['Promotion_Applied'] = promo['Promotion_Applied'].astype(str).str.lower().map({'yes':1,'no':0}).fillna(0).astype(int)
promo_sales = promo.groupby('Promotion_Applied')['Total_Sales'].sum().reset_index()
plt.figure(figsize=(5,4)); plt.bar(['No','Yes'], promo_sales['Total_Sales']); plt.title('Promotion vs Non-Promotion Sales'); plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR,'plot_promo_sales.png')); plt.close()

# 5. Discount vs Units_Sold scatter (sample)
plt.figure(figsize=(6,4)); 
sample = df.sample(n=min(2000,len(df)), random_state=42)
plt.scatter(sample['Discount_Percentage'], sample['Units_Sold'], alpha=0.3, s=10)
plt.xlabel('Discount %'); plt.ylabel('Units Sold'); plt.title('Discount % vs Units Sold (sample)'); plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR,'plot_discount_units.png')); plt.close()

print('EDA plots saved to', OUTPUT_DIR)
monthly_agg.head()

In [ ]:
# Forecasting: create lag features on daily aggregate
daily = daily.set_index('Date').asfreq('D').fillna(0).reset_index()
daily['y'] = daily['Daily_Sales']

# create lag features
for lag in [1,7,14,30]:
    daily[f'lag_{lag}'] = daily['y'].shift(lag).fillna(0)

# rolling means
daily['rmean_7'] = daily['y'].rolling(7,min_periods=1).mean().shift(1).fillna(0)
daily['rmean_30'] = daily['y'].rolling(30,min_periods=1).mean().shift(1).fillna(0)

# add day of week, month
daily['dow'] = daily['Date'].dt.dayofweek
daily['month'] = daily['Date'].dt.month

daily = daily.fillna(0)
daily.tail()

In [ ]:
# Train/test split: last 60 days for testing, rest for training
train = daily.iloc[:-60].copy()
test = daily.iloc[-60:].copy()

FEATURES = [col for col in train.columns if col not in ['Date','y']]
X_train, y_train = train[FEATURES], train['y']
X_test, y_test = test[FEATURES], test['y']

rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=2)
rf.fit(X_train, y_train)
pred_test = rf.predict(X_test)

mae = mean_absolute_error(y_test, pred_test)
rmse = mean_squared_error(y_test, pred_test, squared=False)
print(f'Test MAE: {mae:.2f}, RMSE: {rmse:.2f}')

# Save model
model_path = os.path.join(OUTPUT_DIR, 'rf_daily_sales_model.pkl')
with open(model_path,'wb') as f:
    pickle.dump(rf, f)
print('Saved model to', model_path)

In [ ]:
# Forecast next 30 days using iterative strategy
last_row = daily.iloc[-1:].copy()
future = []
cur = last_row.copy()
for i in range(30):
    X_cur = cur[FEATURES].iloc[0:1]
    y_pred = rf.predict(X_cur)[0]
    next_date = cur['Date'].iloc[0] + pd.Timedelta(days=1)
    new_row = {'Date': next_date, 'y': y_pred}
    # compute new lags/rolling quickly using previous rows: append to daily-like frame
    cur = pd.DataFrame([new_row])
    # compute features based on previous daily with simplistic approach
    prev = daily.append(cur, ignore_index=True).iloc[-31:].copy()
    prev = prev.reset_index(drop=True)
    # build features for the new last row
    row = prev.iloc[-1].copy()
    for lag in [1,7,14,30]:
        lag_val = prev['y'].shift(lag).iloc[-1]
        row[f'lag_{lag}'] = 0 if pd.isna(lag_val) else lag_val
    row['rmean_7'] = prev['y'].rolling(7,min_periods=1).mean().iloc[-2] if len(prev)>=2 else prev['y'].iloc[-1]
    row['rmean_30'] = prev['y'].rolling(30,min_periods=1).mean().iloc[-2] if len(prev)>=2 else prev['y'].iloc[-1]
    row['dow'] = next_date.dayofweek
    row['month'] = next_date.month
    # set cur to this constructed row
    cur = pd.DataFrame([row])
    daily = daily.append(cur, ignore_index=True)

future_forecast = daily.tail(30)[['Date','y']].rename(columns={'y':'forecast'}).reset_index(drop=True)
future_forecast.to_csv(os.path.join(OUTPUT_DIR,'forecast_next_30_days.csv'), index=False)
print('Forecast saved to', os.path.join(OUTPUT_DIR,'forecast_next_30_days.csv'))
future_forecast.head()

In [ ]:
# Simple store segmentation
store_agg = df.groupby('Store_ID').agg(Avg_Sales=('Total_Sales','mean'),
                                       Avg_Discount=('Discount_Percentage','mean'),
                                       Avg_Stock=('Stock_On_Hand','mean')).reset_index().fillna(0)
X_cluster = store_agg[['Avg_Sales','Avg_Discount','Avg_Stock']].copy()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

kmeans = KMeans(n_clusters=3, random_state=42)
store_agg['cluster'] = kmeans.fit_predict(X_scaled)

store_agg.to_csv(os.path.join(OUTPUT_DIR,'store_segmentation.csv'), index=False)
print('Store segmentation saved to', os.path.join(OUTPUT_DIR,'store_segmentation.csv'))
store_agg.head()

In [ ]:
# Create a minimal Streamlit app skeleton (not executed here)
app_code = '''import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
st.set_page_config(layout='wide', page_title='Retail Analytics - Minimal')

st.title('Retail Analytics & Forecasting - Minimal Submission')

@st.cache_data
def load_data():
    df = pd.read_csv('/mnt/data/capstone_outputs/processed_sales_minimal.csv')
    return df

df = load_data()

st.header('Overview')
col1, col2, col3 = st.columns(3)
col1.metric('Total Sales', f"{df['Total_Sales'].sum():,.0f}")
col2.metric('Total Units', f"{df['Units_Sold'].sum():,.0f}")
col3.metric('Unique Stores', df['Store_ID'].nunique())

st.header('Forecast (next 30 days)')
fc = pd.read_csv('/mnt/data/capstone_outputs/forecast_next_30_days.csv')
st.line_chart(fc.set_index('Date')['forecast'])

st.header('Store Segmentation Sample')
seg = pd.read_csv('/mnt/data/capstone_outputs/store_segmentation.csv')
st.dataframe(seg.head(20))
'''

os.makedirs('/mnt/data/capstone_outputs', exist_ok=True)
with open('/mnt/data/dashboard_app_minimal.py','w') as f:
    f.write(app_code)
print('Streamlit app skeleton saved to /mnt/data/dashboard_app_minimal.py')

In [ ]:
# List outputs and show quick samples
import os
out = '/mnt/data/capstone_outputs'
print('Files in output dir:')
print(os.listdir(out))
print('\nForecast sample:')
print(pd.read_csv(os.path.join(out,'forecast_next_30_days.csv')).head())
print('\nStore segmentation sample:')
print(pd.read_csv(os.path.join(out,'store_segmentation.csv')).head())